# Densely Connected Networks (DenseNet)

ResNet significantly changed the view of how to parametrize the functions in deep networks. DenseNet
is to some extent the logical extension of this. To understand how to arrive at it, let’s take a small detour
to theory. Recall the Taylor $3$ expansion for functions. For scalars it can be written as 

$$f(x) = f(0) + f'(x)x + \frac{1}{2} f''(x)x^{2} + \frac{1}{6} f(x)x^{3} + o(x^{3})$$

# Function Decomposition

The key point is that it decomposes the function into increasingly higher order terms. In a similar vein,
ResNet decomposes functions into

                                f(x) = x + g(x)

That is, ResNet decomposes f into a simple linear term and a more complex nonlinear one. What if we
want to go beyond two terms? A solution was proposed by [Huang et al, 2016](https://arxiv.org/abs/1608.06993) in the form of DenseNet, an architecture that reported record performance on the ImageNet dataset.
![](../img/densenet.svg)
Fig. 6.18: The main difference between ResNet (left) and DenseNet (right) in cross-layer connections:
use of addition and use of concatenation.

The key difference between ResNet and DenseNet is that in the latter case outputs are concatenated rather
than added. As a result we perform a mapping from x to its values after applying an increasingly complex
sequence of functions.
            $$x → [x, f 1 (x), f 2 (x, f 1 (x)), f 3 (x, f 1 (x), f 2 (x, f 1 (x)), . . .]$$
            
In the end, all these functions are combined in an MLP to reduce the number of features again. In terms of
implementation this is quite simple - rather than adding terms, we concatenate them. The name DenseNet
arises from the fact that the dependency graph between variables becomes quite dense. The last layer of
such a chain is densely connected to all previous layers. The main components that compose a DenseNet
are dense blocks and transition layers. The former defines how the inputs and outputs are concatenated,
while the latter controls the number of channels so that it is not too large.

![](../img/DenseNetDense.svg)
                           $$ Fig. 6.19: Dense connections in DenseNet $$

# Dense Blocks

DenseNet uses the modified batch normalization, activation, and convolution architecture of ResNet (see
the exercise in the previous section). First, we implement this architecture in the conv_block function.

In [1]:
import sys
sys.path.insert(0, '..')

import d2l
import time
import torch
import torch.nn as nn
import torchvision
from d2l import train_ch5
from torchvision import transforms
from torch.utils.data import DataLoader

def conv_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))
    blk = nn.Sequential(*layers)
    return blk

A dense block consists of multiple conv_block units, each using the same number of output channels.
In the forward computation, however, we concatenate the input and output of each block on the channel
dimension.

In [2]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        layer = []
        for i in range(num_convs):
            layer.append(conv_block((num_channels * i + input_channels), num_channels))
        self.net = nn.Sequential(*layer)
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concatenate the input and output of each block on the channel
            # dimension
            X = torch.cat((X, Y), dim=1)
        return X

In the following example, we define a convolution block with two blocks of 10 output channels. When
using an input with 3 channels, we will get an output with the 3 + 2 × 10 = 23 channels. The number of
convolution block channels controls the increase in the number of output channels relative to the number
of input channels. This is also referred to as the growth rate.

In [3]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

# Transition Layers

Since each dense block will increase the number of channels, adding too many of them will lead to an
excessively complex model. A transition layer is used to control the complexity of the model. It reduces
the number of channels by using the 1 × 1 convolutional layer and halves the height and width of the
average pooling layer with a stride of 2, further reducing the complexity of the model.

In [4]:
def transition_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=1))
    layers.append(nn.AvgPool2d(kernel_size=2, stride=2))
    blk = nn.Sequential(*layers)
    return blk

Apply a transition layer with 10 channels to the output of the dense block in the previous example. This
reduces the number of output channels to 10, and halves the height and width.

In [5]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

# DenseNet Model

Next, we will construct a DenseNet model. DenseNet first uses the same single convolutional layer and
maximum pooling layer as ResNet.

In [6]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1,1,96,96)

layers = []
layers.append(Reshape())
layers.append(nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3))
layers.append(nn.BatchNorm2d(64))
layers.append(nn.ReLU())
layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

Then, similar to the four residual blocks that ResNet uses, DenseNet uses four dense blocks. Similar to
ResNet, we can set the number of convolutional layers used in each dense block. Here, we set it to 4,
consistent with the ResNet-18 in the previous section. Furthermore, we set the number of channels (i.e.
growth rate) for the convolutional layers in the dense block to 32, so 128 channels will be added to each
dense block.

In ResNet, the height and width are reduced between each module by a residual block with a stride of 2.
Here, we use the transition layer to halve the height and width and halve the number of channels.

In [7]:
# Num_channels: the current number of channels
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    layers.append(DenseBlock(num_convs, num_channels, growth_rate))
    # This is the number of output channels in the previous dense block
    num_channels += num_convs * growth_rate
    # A transition layer that haves the number of channels is added between
    # the dense blocks
    if i != len(num_convs_in_dense_blocks) - 1:
        layers.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2
        

Similar to ResNet, a global pooling layer and fully connected layer are connected at the end to produce
the output.

In [8]:
layers.append(nn.BatchNorm2d(num_channels))
layers.append(nn.ReLU())
layers.append(nn.AdaptiveMaxPool2d((1,1)))
layers.append(Flatten())
layers.append(nn.Linear(num_channels, 10))

net = nn.Sequential(*layers)

# Data Acquisition and Training

Since we are using a deeper network here, in this section, we will reduce the input height and width from
224 to 96 to simplify the computation.

In [9]:
lr, num_epochs, batch_size, device = 0.1, 5, 64, d2l.try_gpu()

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        
net.apply(init_weights)
net = net.to(device)
criterion = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
train_ch5(net, train_iter, test_iter, criterion, num_epochs, batch_size, device, lr)

training on cuda:0
